## Autograd

automati differentiation

- Autograd를 사용하면 backprop를 위한 미분 값을 자동으로 계산해 준다.
- 자동 계산을 위해 사용하는 변수는 torch.autograd에 있는 Variable(4.0 부터는 Tensor 그대로도 가능)

## Variable

variable은 data, grad, grad_fn으로 구성되어 있다. 4.0부터는 Tensor가 기본적으로 Variable이 된다.

- data : Tensor 형태의 데이터가 담긴다.
- grad : data가 거쳐온 layer에 대한 미분값이 축적된다.
- grad_fn : 미분 값을 계산한 함수에 대한 정보

requires_grad가 True인 tensor가 Function의 결과로 생성된 tensor는 grad_fn를 가지고 있다.    
사용자가 직접 생성한 tensor는 grad_fn를 가지고 있지 않다. 직접 생성한 Tensor는 직접적으로 연산하지 않으므로, 기울기를 계산할 필요가 없기 때문이다.

In [1]:
import torch
from torch.autograd import Variable

In [2]:
a = torch.ones(2, 2, requires_grad=True) #requires_grad를 설정해 기울기를 계산해야 하는 값임을 알려준다.
a

#이전에는 Tensor를 Variable로 변환해서 연산에 사용하고, requires_grad 도 설정해줘야 했지만, 
#4.0 이후 부터는 Tensor에서 바로 설정하고 연산에 적용할 수 있다.
#Pytorch_Tutorials/Deep_Learning_with_PyTorch-A_60_Minute_Blitz/02_autograd_tutorial.ipynb 참고

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [3]:
print(a.data)
print(a.grad)
print(a.grad_fn)

tensor([[1., 1.],
        [1., 1.]])
None
None


In [4]:
b = a + 2
b

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)

In [5]:
c = b**2
c

tensor([[9., 9.],
        [9., 9.]], grad_fn=<PowBackward0>)

In [6]:
out = c.sum()
out

tensor(36., grad_fn=<SumBackward0>)

In [7]:
out.backward() #역전파 한다. # 두 번 호출하면 다른 값이 나오거나 오류 발생할 수 있다.
#메모리 최적화를 위해, 더 이상 필요하지 않은 값은 삭제해 버리기 때문

#requires_grad가 True인 tensor가 Function의 결과로 생성된 tensor는 grad_fn를 가지고 있다.

In [8]:
print(a.data)
print(a.grad)
print(a.grad_fn) #사용자가 직접 생성한 tensor는 grad_fn를 가지고 있지 않다. 
#직접 생성한 Tensor는 직접적으로 연산하지 않으므로, 기울기를 계산할 필요가 없기 때문이다.

#이전과 달리, grad의 값이 생긴다.

tensor([[1., 1.],
        [1., 1.]])
tensor([[6., 6.],
        [6., 6.]])
None


In [9]:
print(b.data)
print(b.grad)
print(b.grad_fn) #requires_grad가 True인 tensor(a)가 Function의 결과로 생성된 tensor는 grad_fn를 가지고 있다.
#b = a + 2 에 대한 역전파를 할 때의 grad_fn가 담긴다.

tensor([[3., 3.],
        [3., 3.]])
None


In [10]:
print(c.data)
print(c.grad)
print(c.grad_fn) #requires_grad가 True인 tensor(a)가 Function의 결과로 생성된 tensor는 grad_fn를 가지고 있다.
#b = b^2 에 대한 역전파를 할 때의 grad_fn가 담긴다.

tensor([[9., 9.],
        [9., 9.]])
None


In [11]:
print(out.data)
print(out.grad)
print(out.grad_fn) #requires_grad가 True인 tensor(a)가 Function의 결과로 생성된 tensor는 grad_fn를 가지고 있다.
#out = c.sum() 에 대한 역전파를 할 때의 grad_fn가 담긴다.

tensor(36.)
None


## 주의할 점

In [12]:
x = torch.ones(3, requires_grad=True)
y = x**2
z = y * 3
print(z)

tensor([3., 3., 3.], grad_fn=<MulBackward>)


In [13]:
grad = torch.Tensor([0.1, 1, 10])
z.backward(grad)
#z = 3 * x^2 이므로 이를 미분하면 3 * 2 *x(1) 이므로 6이 된다.

In [14]:
print(x.data)
print(x.grad)
print(x.grad_fn)

#하지만, grad에서는 [ 0.6000,  6.0000, 60.0000]이 나오는데, 이는 backward 매개변수로 넣어진 grad의 형태에 맞춰서 곱해지기 때문이다.

tensor([1., 1., 1.])
tensor([ 0.6000,  6.0000, 60.0000])
None
